In [2]:
import pandas as pd
import re
from pandas.io.json import json_normalize
from sqlalchemy import create_engine

disk_engine = create_engine('sqlite:///db/livefyre.db')

In [100]:
df_comments_raw = pd.read_csv('livefyre_raw/df_comments_all.csv',sep=';',encoding='utf-8')
df_title_raw_1 = pd.read_csv('livefyre_raw/df_all_titles.csv',sep=';',encoding='utf-8')


C:\Users\ClementLefevre\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [107]:
df_title_raw_2 = pd.read_csv('df_all_titles_386.csv',sep=';',encoding='utf-8')
df_title_raw_2['ft_id']= df_title_raw_2.ft_id.str.extract('(?<=\/)(.*?)(?=\?|\#)')

df_title_raw_2.ft_id.values

C:\Users\ClementLefevre\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  


array(['/www.ft.com/content/016a1ede-6059-11e6-b38c-7b39cbb1138a',
       '/www.ft.com/content/016b5f74-975b-11e4-be9d-00144feabdc0',
       '/www.ft.com/content/01c658c8-c251-11e3-9370-00144feabdc0',
       '/www.ft.com/content/020e1078-8400-11e6-a29c-6e7d9515ad15',
       '/www.ft.com/content/0253717e-598d-11e6-8d05-4eaa66292c32',
       '/www.ft.com/content/02796d92-94dc-11e4-8341-00144feabdc0',
       '/www.ft.com/content/02a46e28-4816-11e5-b3b2-1672f710807b',
       '/www.ft.com/content/032a8e84-ea2e-11e6-893c-082c54a7f539',
       '/next.ft.com/content/03d3256c-2bb7-11e5-8613-e7aedbb7bdb7',
       '/www.ft.com/content/0409a032-c56b-11e3-a7d4-00144feabdc0',
       '/cncc.bingj.com/cache.aspx', nan,
       '/www.ft.com/content/080a7a4c-9a7c-11e6-b8c6-568a43813464',
       '/www.ft.com/content/095a6c48-9461-11e6-a80e-bcd69f323a8b',
       '/www.ft.com/content/0a400a7a-afbd-11e5-b955-1a1d298b6250',
       '/www.ft.com/content/0a4ca8de-bf9d-11e6-9bca-2b93a6856354',
       '/www.ft.com

In [110]:
df_title_raw_2[df_title_raw_2.ft_id.isnull()].values

array([[11, nan, 'How China rules the waves — FT.com'],
       [103, nan, 'Saudi Aramco IPO valuation calculator — FT.com']],
      dtype=object)

In [111]:
df_title_raw = pd.concat([df_title_raw_1,df_title_raw_2],axis=0)

In [112]:
def extract_ft_id(row):
    return row.split('/')[-1].split('.')[0]

In [113]:
df_title_raw[df_title_raw.ft_id.isnull()]

,Unnamed: 0,Unnamed: 0.1,ft_id,title
612,612,612.0,NaN,How China rules the waves — FT.com
8217,8217,8217.0,NaN,Saudi Aramco IPO valuation calculator — FT.com
12433,12433,12433.0,NaN,Fighting a dictator doesn’t make one a freedom...
12449,12449,12449.0,NaN,"Oil prices sink, Brazil’s perilous transition ..."
27554,13,13.0,NaN,How China rules the waves — FT.com
27652,111,111.0,NaN,Saudi Aramco IPO valuation calculator — FT.com
11,11,NaN,NaN,How China rules the waves — FT.com
103,103,NaN,NaN,Saudi Aramco IPO valuation calculator — FT.com


In [115]:
df_title_raw = df_title_raw[~df_title_raw.ft_id.isnull()]

In [116]:
df_title_raw['ft_id']= df_title_raw.ft_id.apply(extract_ft_id)

In [117]:
df_title_raw.head()

,Unnamed: 0,Unnamed: 0.1,ft_id,title
0,0,0.0,000137ba-1eb6-11e6-b286-cddde55ca122,Fledgling emerging market reprieve ends as Fed...
1,1,1.0,0002f9ce-214d-11e8-9efc-0cd3483b8b80,UK retailers count cost of ‘beast from the eas...
2,2,2.0,0007a1c4-4fcf-11e5-8642-453585f2cfcd,The moral hazard created by buckets of start-u...
3,3,3.0,00096d84-7453-11e6-bf48-b372cdb1043a,Wood Group close to resolving North Sea worker...
4,4,4.0,0009bce4-a568-11e6-8b69-02899e8bd9d1,Cargill seeks out fertile ground in Asia front...


In [118]:
print (df_comments_raw.shape)
print (df_title_raw.shape)

(525727, 19)
(28574, 4)


In [119]:
df_comment_merged = pd.merge(df_comments_raw.drop(['title'],axis=1),df_title_raw, on ='ft_id', how='left' )

In [120]:
print (df_comment_merged.shape)
df_comment_merged.head(2)

(525892, 21)


,0,ancestorId,annotations,attachments,authorId,bodyHtml,createdAt,featuredmessage,ft_id,generator,...,likedBy,links,messagetag,moderator,parentId,updatedAt,updatedBy,Unnamed: 0,Unnamed: 0.1,title
0,NaN,NaN,"{'messagetag': ['spam-1', 'toxic-1'], 'links':...",NaN,8719926@ft.fyre.co,<p>Whatever people needed to buy last week and...,1.520376e+09,NaN,0002f9ce-214d-11e8-9efc-0cd3483b8b80,{'id': 'livefyre.com'},...,NaN,{},"['spam-1', 'toxic-1']",NaN,NaN,1.520376e+09,NaN,1.0,1.0,UK retailers count cost of ‘beast from the eas...
1,NaN,778299845.0,"{'likedBy': ['12570357@ft.fyre.co'], 'messaget...",NaN,3684198@ft.fyre.co,"<p>Hmmm, not so sure I agree that’s true for a...",1.520401e+09,NaN,0002f9ce-214d-11e8-9efc-0cd3483b8b80,{'id': 'livefyre.com'},...,['12570357@ft.fyre.co'],{},"['spam-0', 'toxic-1', 'moderated']",NaN,778299845.0,1.520401e+09,NaN,1.0,1.0,UK retailers count cost of ‘beast from the eas...


In [122]:
#df_comment_merged.to_csv('livefyre_raw/df_comments_all_with_title.csv',sep=';',encoding='utf-8')